In [ ]:
%pip install geopandas

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# If the files have been named from QGIS in a consistent format of Count_[fatality]_[location].csv, they can be more easily accessed this way
fatality = "Fatal"
route = "OnRoute"

type = "_" + str(fatality) + "_" + str(route)

In [ ]:
data = pd.read_csv("Count" + type + ".csv")

In [ ]:
# Get the area and congestion values for each region
areaData = pd.read_csv("Region_Area_Values.csv")
congestData = pd.read_csv("Avg_Congestions.csv")

In [ ]:
# Normalize the area and congestion data
areaData = (areaData - areaData.min()) / (areaData.max() - areaData.min()) + 1

congestData = (congestData - congestData.min()) / (congestData.max() - congestData.min()) + 1

In [ ]:
congestData

In [ ]:
crashesTotal = {"REGION_ID": range(1,30), "CRASH_RATIO": np.zeros((29),dtype=int)}
crashesCongestion = {"REGION_ID": range(1,30), "CRASH_RATIO": np.zeros((29),dtype=int)}
crashesEBE = {"REGION_ID": range(1,30), "CRASH_RATIO": np.zeros((29),dtype=int)}

In [ ]:
# Historical Congestion Estimate
for i in range(29):
    crashesTotal["CRASH_RATIO"][i] = (data["NUMPOINTS"][i] + 0.0) / areaData["AREA"][i]

    crashesCongestion["CRASH_RATIO"][i] = (data["NUMPOINTS"][i] + 0.0) / areaData["AREA"][i]
    crashesCongestion["CRASH_RATIO"][i] = crashesCongestion["CRASH_RATIO"][i] / congestData["AVG_CONGESTION"][i]

    crashesEBE["CRASH_RATIO"][i] = (data["NUMPOINTS"][i] + 0.0) / areaData["AREA"][i]

In [ ]:
crashesCongestion

In [ ]:
# Manually defined list of which regions touch which regions based on the visual map of the 29 regions
touching = {1: [3,4],
            2: [3,5,6],
            3: [1,2,4,5,6,8],
            4: [1,3,6,8],
            5: [2,3,6,7,9,10],
            6: [2,3,4,5,6,8],
            7: [5,6,8,9,10,11],
            8: [3,4,6,7,11,12,29],
            9: [5,7,10,14],
            10: [5,7,9,11,14],
            11: [7,8,10,12,13,14,15],
            12: [8,11,13,29],
            13: [11,12,15,16,29],
            14: [9,10,11,15,17,18],
            15: [11,13,14,16,18,19,20],
            16: [13,15,19,20,21,29],
            17: [14,18,22],
            18: [14,15,17,19,22],
            19: [15,16,18,20,22,23],
            20: [15,16,19,21,23,24],
            21: [16,20,23,24],
            22: [17,18,19,23,25],
            23: [19,20,21,22,24,25,26],
            24: [20,21,23,26,27],
            25: [22,23,26,28],
            26: [23,24,25,27,28],
            27: [24,26,28],
            28: [25,26,27],
            29: [8,12,23,16]
            }

In [ ]:
# Empirical Bayes Estimate
for i in range(29):
    realValue = crashesEBE["CRASH_RATIO"][i]

    values = []
    for j in touching[i + 1]:
        values.append(crashesEBE["CRASH_RATIO"][j - 1])

    expFreq = np.average(np.array(values))
    weight = 0
    if (expFreq + np.var(np.array(values))) != 0:
        weight = expFreq / (expFreq + np.var(np.array(values)))

    crashesEBE["CRASH_RATIO"][i] = (weight * expFreq) + ((1 - weight) * realValue)

In [ ]:
# Save CSVs of the final results
pd.DataFrame(crashesTotal).to_csv("Results_Base" + type + ".csv", index=False)
pd.DataFrame(crashesCongestion).to_csv("Results_Congest" + type + ".csv", index=False)
pd.DataFrame(crashesEBE).to_csv("Results_EBE" + type + ".csv", index=False)

In [ ]:
# Normalize the data for the figures
normalTotalData = (pd.DataFrame(crashesTotal) - pd.DataFrame(crashesTotal).min()) / (pd.DataFrame(crashesTotal).max() - pd.DataFrame(crashesTotal).min()) * 0.8 + 0.2
if ((pd.DataFrame(crashesCongestion).max() - pd.DataFrame(crashesCongestion).min())["CRASH_RATIO"] != 0):
    normalCongestionData = (pd.DataFrame(crashesCongestion) - pd.DataFrame(crashesCongestion).min()) / (pd.DataFrame(crashesCongestion).max() - pd.DataFrame(crashesCongestion).min()) * 0.8 + 0.2
else:
    normalCongestionData = pd.DataFrame(crashesCongestion) + 0.2
normalEBEData = (pd.DataFrame(crashesEBE) - pd.DataFrame(crashesEBE).min()) / (pd.DataFrame(crashesEBE).max() - pd.DataFrame(crashesEBE).min()) * 0.8 + 0.2

In [ ]:
%pip install matplotlib

In [ ]:
regionData = pd.read_csv("TrafficRegions.csv")

In [ ]:
regionData["BOUNDARY"][0]

In [ ]:
# Draw each region colored by the total crash data
import matplotlib.pyplot as plt

region1 = plt.Polygon([(-87.709645,42.026444),(-87.654561,42.026444),(-87.654561,41.997946),(-87.709645,41.997946)], facecolor='red', alpha=normalTotalData["CRASH_RATIO"][0])

region2 = plt.Polygon([(-87.84621,42.0190998),(-87.747456,42.0190998),(-87.747456,41.960669),(-87.84621,41.960669)], facecolor='red', alpha=normalTotalData["CRASH_RATIO"][1])

region3 = plt.Polygon([(-87.747456,41.997946),(-87.67459,41.997946),(-87.67459,41.960669),(-87.747456,41.960669)], facecolor='red', alpha=normalTotalData["CRASH_RATIO"][2])

region4 = plt.Polygon([(-87.67459,41.997946),(-87.646438,41.997946),(-87.646438,41.960669),(-87.67459,41.960669)], facecolor='red', alpha=normalTotalData["CRASH_RATIO"][3])

region5 = plt.Polygon([(-87.84621,41.960669),(-87.747456,41.960669),(-87.747456,41.909269),(-87.84621,41.909269)], facecolor='red', alpha=normalTotalData["CRASH_RATIO"][4])

region6 = plt.Polygon([(-87.747456,41.960669),(-87.67459,41.960669),(-87.67459,41.931841),(-87.747456,41.931841)], facecolor='red', alpha=normalTotalData["CRASH_RATIO"][5])

region7 = plt.Polygon([(-87.747456,41.931841),(-87.67459,41.931841),(-87.67459,41.909269),(-87.747456,41.909269)], facecolor='red', alpha=normalTotalData["CRASH_RATIO"][6])

region8 = plt.Polygon([(-87.67459,41.960669),(-87.619112,41.960669),(-87.619112,41.910561),(-87.67459,41.910561)], facecolor='red', alpha=normalTotalData["CRASH_RATIO"][7])

region9 = plt.Polygon([(-87.775947,41.909269),(-87.747456,41.909269),(-87.747456,41.866129),(-87.775947,41.866129)], facecolor='red', alpha=normalTotalData["CRASH_RATIO"][8])

region10 = plt.Polygon([(-87.747456,41.909269),(-87.691617,41.909269),(-87.691617,41.866129),(-87.747456,41.866129)], facecolor='red', alpha=normalTotalData["CRASH_RATIO"][9])

region11 = plt.Polygon([(-87.691617,41.910561),(-87.647208,41.910561),(-87.647208,41.866129),(-87.691617,41.866129)], facecolor='red', alpha=normalTotalData["CRASH_RATIO"][10])

region12 = plt.Polygon([(-87.647208,41.911401),(-87.62308,41.911401),(-87.62308,41.88886),(-87.647208,41.88886)], facecolor='red', alpha=normalTotalData["CRASH_RATIO"][11])

region13 = plt.Polygon([(-87.647208,41.88886),(-87.62308,41.88886),(-87.62308,41.866129),(-87.647208,41.866129)], facecolor='red', alpha=normalTotalData["CRASH_RATIO"][12])

region14 = plt.Polygon([(-87.747456,41.866129),(-87.685372,41.866129),(-87.685372,41.822792),(-87.747456,41.822792)], facecolor='red', alpha=normalTotalData["CRASH_RATIO"][13])

region15 = plt.Polygon([(-87.685372,41.866129),(-87.636322,41.866129),(-87.636322,41.822792),(-87.685372,41.822792)], facecolor='red', alpha=normalTotalData["CRASH_RATIO"][14])

region16 = plt.Polygon([(-87.636322,41.866129),(-87.597952,41.866129),(-87.597952,41.822792),(-87.636322,41.822792)], facecolor='red', alpha=normalTotalData["CRASH_RATIO"][15])

region17 = plt.Polygon([(-87.802188,41.822792),(-87.747456,41.822792),(-87.747456,41.764066),(-87.802188,41.764066)], facecolor='red', alpha=normalTotalData["CRASH_RATIO"][16])

region18 = plt.Polygon([(-87.747456,41.822792),(-87.68373,41.822792),(-87.68373,41.764066),(-87.747456,41.764066)], facecolor='red', alpha=normalTotalData["CRASH_RATIO"][17])

region19 = plt.Polygon([(-87.68373,41.822792),(-87.636322,41.822792),(-87.636322,41.764066),(-87.68373,41.764066)], facecolor='red', alpha=normalTotalData["CRASH_RATIO"][18])

region20 = plt.Polygon([(-87.636322,41.822792),(-87.606334,41.822792),(-87.606334,41.764066),(-87.636322,41.764066)], facecolor='red', alpha=normalTotalData["CRASH_RATIO"][19])

region21 = plt.Polygon([(-87.606334,41.822792),(-87.56626,41.822792),(-87.56626,41.764066),(-87.606334,41.764066)], facecolor='red', alpha=normalTotalData["CRASH_RATIO"][20])

region22 = plt.Polygon([(-87.747456,41.764066),(-87.67298,41.764066),(-87.67298,41.728472),(-87.747456,41.728472)], facecolor='red', alpha=normalTotalData["CRASH_RATIO"][21])

region23 = plt.Polygon([(-87.67298,41.764066),(-87.606334,41.764066),(-87.606334,41.728472),(-87.67298,41.728472)], facecolor='red', alpha=normalTotalData["CRASH_RATIO"][22])

region24 = plt.Polygon([(-87.606334,41.764066),(-87.526388,41.764066),(-87.526388,41.728472),(-87.606334,41.728472)], facecolor='red', alpha=normalTotalData["CRASH_RATIO"][23])

region25 = plt.Polygon([(-87.747456,41.728472),(-87.662508,41.728472),(-87.662508,41.677477),(-87.747456,41.677477)], facecolor='red', alpha=normalTotalData["CRASH_RATIO"][24])

region26 = plt.Polygon([(-87.662508,41.728472),(-87.584845,41.728472),(-87.584845,41.677477),(-87.662508,41.677477)], facecolor='red', alpha=normalTotalData["CRASH_RATIO"][25])

region27 = plt.Polygon([(-87.584845,41.728472),(-87.524436,41.728472),(-87.524436,41.677477),(-87.584845,41.677477)], facecolor='red', alpha=normalTotalData["CRASH_RATIO"][26])

region28 = plt.Polygon([(-87.662508,41.677477),(-87.524436,41.677477),(-87.524436,41.644139),(-87.662508,41.644139)], facecolor='red', alpha=normalTotalData["CRASH_RATIO"][27])

region29 = plt.Polygon([(-87.62308,41.911401),(-87.595378,41.911401),(-87.595378,41.866129),(-87.62308,41.866129)], facecolor='red', alpha=normalTotalData["CRASH_RATIO"][28])


fig, ax = plt.subplots(1)

plt.xlim(-87.9,-87.5)
plt.ylim(41.6,42.05)

ax.add_patch(region1)
ax.add_patch(region2)
ax.add_patch(region3)
ax.add_patch(region4)
ax.add_patch(region5)
ax.add_patch(region6)
ax.add_patch(region7)
ax.add_patch(region8)
ax.add_patch(region9)
ax.add_patch(region10)
ax.add_patch(region11)
ax.add_patch(region12)
ax.add_patch(region13)
ax.add_patch(region14)
ax.add_patch(region15)
ax.add_patch(region16)
ax.add_patch(region17)
ax.add_patch(region18)
ax.add_patch(region19)
ax.add_patch(region20)
ax.add_patch(region21)
ax.add_patch(region22)
ax.add_patch(region23)
ax.add_patch(region24)
ax.add_patch(region25)
ax.add_patch(region26)
ax.add_patch(region27)
ax.add_patch(region28)
ax.add_patch(region29)


ax.set_aspect('equal', adjustable='box')
ax.set_axis_off()

plt.title("Crashes by Area \nfor " + fatality + " / " + route)

plt.savefig("Fig_Base" + type + ".png", bbox_inches='tight')
plt.show()

In [ ]:
# Draw each region colored by the congestion estimate data
import matplotlib.pyplot as plt

region1 = plt.Polygon([(-87.709645,42.026444),(-87.654561,42.026444),(-87.654561,41.997946),(-87.709645,41.997946)], facecolor='red', alpha=normalCongestionData["CRASH_RATIO"][0])

region2 = plt.Polygon([(-87.84621,42.0190998),(-87.747456,42.0190998),(-87.747456,41.960669),(-87.84621,41.960669)], facecolor='red', alpha=normalCongestionData["CRASH_RATIO"][1])

region3 = plt.Polygon([(-87.747456,41.997946),(-87.67459,41.997946),(-87.67459,41.960669),(-87.747456,41.960669)], facecolor='red', alpha=normalCongestionData["CRASH_RATIO"][2])

region4 = plt.Polygon([(-87.67459,41.997946),(-87.646438,41.997946),(-87.646438,41.960669),(-87.67459,41.960669)], facecolor='red', alpha=normalCongestionData["CRASH_RATIO"][3])

region5 = plt.Polygon([(-87.84621,41.960669),(-87.747456,41.960669),(-87.747456,41.909269),(-87.84621,41.909269)], facecolor='red', alpha=normalCongestionData["CRASH_RATIO"][4])

region6 = plt.Polygon([(-87.747456,41.960669),(-87.67459,41.960669),(-87.67459,41.931841),(-87.747456,41.931841)], facecolor='red', alpha=normalCongestionData["CRASH_RATIO"][5])

region7 = plt.Polygon([(-87.747456,41.931841),(-87.67459,41.931841),(-87.67459,41.909269),(-87.747456,41.909269)], facecolor='red', alpha=normalCongestionData["CRASH_RATIO"][6])

region8 = plt.Polygon([(-87.67459,41.960669),(-87.619112,41.960669),(-87.619112,41.910561),(-87.67459,41.910561)], facecolor='red', alpha=normalCongestionData["CRASH_RATIO"][7])

region9 = plt.Polygon([(-87.775947,41.909269),(-87.747456,41.909269),(-87.747456,41.866129),(-87.775947,41.866129)], facecolor='red', alpha=normalCongestionData["CRASH_RATIO"][8])

region10 = plt.Polygon([(-87.747456,41.909269),(-87.691617,41.909269),(-87.691617,41.866129),(-87.747456,41.866129)], facecolor='red', alpha=normalCongestionData["CRASH_RATIO"][9])

region11 = plt.Polygon([(-87.691617,41.910561),(-87.647208,41.910561),(-87.647208,41.866129),(-87.691617,41.866129)], facecolor='red', alpha=normalCongestionData["CRASH_RATIO"][10])

region12 = plt.Polygon([(-87.647208,41.911401),(-87.62308,41.911401),(-87.62308,41.88886),(-87.647208,41.88886)], facecolor='red', alpha=normalCongestionData["CRASH_RATIO"][11])

region13 = plt.Polygon([(-87.647208,41.88886),(-87.62308,41.88886),(-87.62308,41.866129),(-87.647208,41.866129)], facecolor='red', alpha=normalCongestionData["CRASH_RATIO"][12])

region14 = plt.Polygon([(-87.747456,41.866129),(-87.685372,41.866129),(-87.685372,41.822792),(-87.747456,41.822792)], facecolor='red', alpha=normalCongestionData["CRASH_RATIO"][13])

region15 = plt.Polygon([(-87.685372,41.866129),(-87.636322,41.866129),(-87.636322,41.822792),(-87.685372,41.822792)], facecolor='red', alpha=normalCongestionData["CRASH_RATIO"][14])

region16 = plt.Polygon([(-87.636322,41.866129),(-87.597952,41.866129),(-87.597952,41.822792),(-87.636322,41.822792)], facecolor='red', alpha=normalCongestionData["CRASH_RATIO"][15])

region17 = plt.Polygon([(-87.802188,41.822792),(-87.747456,41.822792),(-87.747456,41.764066),(-87.802188,41.764066)], facecolor='red', alpha=normalCongestionData["CRASH_RATIO"][16])

region18 = plt.Polygon([(-87.747456,41.822792),(-87.68373,41.822792),(-87.68373,41.764066),(-87.747456,41.764066)], facecolor='red', alpha=normalCongestionData["CRASH_RATIO"][17])

region19 = plt.Polygon([(-87.68373,41.822792),(-87.636322,41.822792),(-87.636322,41.764066),(-87.68373,41.764066)], facecolor='red', alpha=normalCongestionData["CRASH_RATIO"][18])

region20 = plt.Polygon([(-87.636322,41.822792),(-87.606334,41.822792),(-87.606334,41.764066),(-87.636322,41.764066)], facecolor='red', alpha=normalCongestionData["CRASH_RATIO"][19])

region21 = plt.Polygon([(-87.606334,41.822792),(-87.56626,41.822792),(-87.56626,41.764066),(-87.606334,41.764066)], facecolor='red', alpha=normalCongestionData["CRASH_RATIO"][20])

region22 = plt.Polygon([(-87.747456,41.764066),(-87.67298,41.764066),(-87.67298,41.728472),(-87.747456,41.728472)], facecolor='red', alpha=normalCongestionData["CRASH_RATIO"][21])

region23 = plt.Polygon([(-87.67298,41.764066),(-87.606334,41.764066),(-87.606334,41.728472),(-87.67298,41.728472)], facecolor='red', alpha=normalCongestionData["CRASH_RATIO"][22])

region24 = plt.Polygon([(-87.606334,41.764066),(-87.526388,41.764066),(-87.526388,41.728472),(-87.606334,41.728472)], facecolor='red', alpha=normalCongestionData["CRASH_RATIO"][23])

region25 = plt.Polygon([(-87.747456,41.728472),(-87.662508,41.728472),(-87.662508,41.677477),(-87.747456,41.677477)], facecolor='red', alpha=normalCongestionData["CRASH_RATIO"][24])

region26 = plt.Polygon([(-87.662508,41.728472),(-87.584845,41.728472),(-87.584845,41.677477),(-87.662508,41.677477)], facecolor='red', alpha=normalCongestionData["CRASH_RATIO"][25])

region27 = plt.Polygon([(-87.584845,41.728472),(-87.524436,41.728472),(-87.524436,41.677477),(-87.584845,41.677477)], facecolor='red', alpha=normalCongestionData["CRASH_RATIO"][26])

region28 = plt.Polygon([(-87.662508,41.677477),(-87.524436,41.677477),(-87.524436,41.644139),(-87.662508,41.644139)], facecolor='red', alpha=normalCongestionData["CRASH_RATIO"][27])

region29 = plt.Polygon([(-87.62308,41.911401),(-87.595378,41.911401),(-87.595378,41.866129),(-87.62308,41.866129)], facecolor='red', alpha=normalCongestionData["CRASH_RATIO"][28])


fig, ax = plt.subplots(1)

plt.xlim(-87.9,-87.5)
plt.ylim(41.6,42.05)

ax.add_patch(region1)
ax.add_patch(region2)
ax.add_patch(region3)
ax.add_patch(region4)
ax.add_patch(region5)
ax.add_patch(region6)
ax.add_patch(region7)
ax.add_patch(region8)
ax.add_patch(region9)
ax.add_patch(region10)
ax.add_patch(region11)
ax.add_patch(region12)
ax.add_patch(region13)
ax.add_patch(region14)
ax.add_patch(region15)
ax.add_patch(region16)
ax.add_patch(region17)
ax.add_patch(region18)
ax.add_patch(region19)
ax.add_patch(region20)
ax.add_patch(region21)
ax.add_patch(region22)
ax.add_patch(region23)
ax.add_patch(region24)
ax.add_patch(region25)
ax.add_patch(region26)
ax.add_patch(region27)
ax.add_patch(region28)
ax.add_patch(region29)


ax.set_aspect('equal', adjustable='box')
ax.set_axis_off()

plt.title("Crashes by Area per \nTraffic Congestion for " + fatality + " / " + route)

plt.savefig("Fig_Congest" + type + ".png", bbox_inches='tight')
plt.show()

In [ ]:
# Draw each region colored by the EB-Estimate data
import matplotlib.pyplot as plt

region1 = plt.Polygon([(-87.709645,42.026444),(-87.654561,42.026444),(-87.654561,41.997946),(-87.709645,41.997946)], facecolor='red', alpha=normalEBEData["CRASH_RATIO"][0])

region2 = plt.Polygon([(-87.84621,42.0190998),(-87.747456,42.0190998),(-87.747456,41.960669),(-87.84621,41.960669)], facecolor='red', alpha=normalEBEData["CRASH_RATIO"][1])

region3 = plt.Polygon([(-87.747456,41.997946),(-87.67459,41.997946),(-87.67459,41.960669),(-87.747456,41.960669)], facecolor='red', alpha=normalEBEData["CRASH_RATIO"][2])

region4 = plt.Polygon([(-87.67459,41.997946),(-87.646438,41.997946),(-87.646438,41.960669),(-87.67459,41.960669)], facecolor='red', alpha=normalEBEData["CRASH_RATIO"][3])

region5 = plt.Polygon([(-87.84621,41.960669),(-87.747456,41.960669),(-87.747456,41.909269),(-87.84621,41.909269)], facecolor='red', alpha=normalEBEData["CRASH_RATIO"][4])

region6 = plt.Polygon([(-87.747456,41.960669),(-87.67459,41.960669),(-87.67459,41.931841),(-87.747456,41.931841)], facecolor='red', alpha=normalEBEData["CRASH_RATIO"][5])

region7 = plt.Polygon([(-87.747456,41.931841),(-87.67459,41.931841),(-87.67459,41.909269),(-87.747456,41.909269)], facecolor='red', alpha=normalEBEData["CRASH_RATIO"][6])

region8 = plt.Polygon([(-87.67459,41.960669),(-87.619112,41.960669),(-87.619112,41.910561),(-87.67459,41.910561)], facecolor='red', alpha=normalEBEData["CRASH_RATIO"][7])

region9 = plt.Polygon([(-87.775947,41.909269),(-87.747456,41.909269),(-87.747456,41.866129),(-87.775947,41.866129)], facecolor='red', alpha=normalEBEData["CRASH_RATIO"][8])

region10 = plt.Polygon([(-87.747456,41.909269),(-87.691617,41.909269),(-87.691617,41.866129),(-87.747456,41.866129)], facecolor='red', alpha=normalEBEData["CRASH_RATIO"][9])

region11 = plt.Polygon([(-87.691617,41.910561),(-87.647208,41.910561),(-87.647208,41.866129),(-87.691617,41.866129)], facecolor='red', alpha=normalEBEData["CRASH_RATIO"][10])

region12 = plt.Polygon([(-87.647208,41.911401),(-87.62308,41.911401),(-87.62308,41.88886),(-87.647208,41.88886)], facecolor='red', alpha=normalEBEData["CRASH_RATIO"][11])

region13 = plt.Polygon([(-87.647208,41.88886),(-87.62308,41.88886),(-87.62308,41.866129),(-87.647208,41.866129)], facecolor='red', alpha=normalEBEData["CRASH_RATIO"][12])

region14 = plt.Polygon([(-87.747456,41.866129),(-87.685372,41.866129),(-87.685372,41.822792),(-87.747456,41.822792)], facecolor='red', alpha=normalEBEData["CRASH_RATIO"][13])

region15 = plt.Polygon([(-87.685372,41.866129),(-87.636322,41.866129),(-87.636322,41.822792),(-87.685372,41.822792)], facecolor='red', alpha=normalEBEData["CRASH_RATIO"][14])

region16 = plt.Polygon([(-87.636322,41.866129),(-87.597952,41.866129),(-87.597952,41.822792),(-87.636322,41.822792)], facecolor='red', alpha=normalEBEData["CRASH_RATIO"][15])

region17 = plt.Polygon([(-87.802188,41.822792),(-87.747456,41.822792),(-87.747456,41.764066),(-87.802188,41.764066)], facecolor='red', alpha=normalEBEData["CRASH_RATIO"][16])

region18 = plt.Polygon([(-87.747456,41.822792),(-87.68373,41.822792),(-87.68373,41.764066),(-87.747456,41.764066)], facecolor='red', alpha=normalEBEData["CRASH_RATIO"][17])

region19 = plt.Polygon([(-87.68373,41.822792),(-87.636322,41.822792),(-87.636322,41.764066),(-87.68373,41.764066)], facecolor='red', alpha=normalEBEData["CRASH_RATIO"][18])

region20 = plt.Polygon([(-87.636322,41.822792),(-87.606334,41.822792),(-87.606334,41.764066),(-87.636322,41.764066)], facecolor='red', alpha=normalEBEData["CRASH_RATIO"][19])

region21 = plt.Polygon([(-87.606334,41.822792),(-87.56626,41.822792),(-87.56626,41.764066),(-87.606334,41.764066)], facecolor='red', alpha=normalEBEData["CRASH_RATIO"][20])

region22 = plt.Polygon([(-87.747456,41.764066),(-87.67298,41.764066),(-87.67298,41.728472),(-87.747456,41.728472)], facecolor='red', alpha=normalEBEData["CRASH_RATIO"][21])

region23 = plt.Polygon([(-87.67298,41.764066),(-87.606334,41.764066),(-87.606334,41.728472),(-87.67298,41.728472)], facecolor='red', alpha=normalEBEData["CRASH_RATIO"][22])

region24 = plt.Polygon([(-87.606334,41.764066),(-87.526388,41.764066),(-87.526388,41.728472),(-87.606334,41.728472)], facecolor='red', alpha=normalEBEData["CRASH_RATIO"][23])

region25 = plt.Polygon([(-87.747456,41.728472),(-87.662508,41.728472),(-87.662508,41.677477),(-87.747456,41.677477)], facecolor='red', alpha=normalEBEData["CRASH_RATIO"][24])

region26 = plt.Polygon([(-87.662508,41.728472),(-87.584845,41.728472),(-87.584845,41.677477),(-87.662508,41.677477)], facecolor='red', alpha=normalEBEData["CRASH_RATIO"][25])

region27 = plt.Polygon([(-87.584845,41.728472),(-87.524436,41.728472),(-87.524436,41.677477),(-87.584845,41.677477)], facecolor='red', alpha=normalEBEData["CRASH_RATIO"][26])

region28 = plt.Polygon([(-87.662508,41.677477),(-87.524436,41.677477),(-87.524436,41.644139),(-87.662508,41.644139)], facecolor='red', alpha=normalEBEData["CRASH_RATIO"][27])

region29 = plt.Polygon([(-87.62308,41.911401),(-87.595378,41.911401),(-87.595378,41.866129),(-87.62308,41.866129)], facecolor='red', alpha=normalEBEData["CRASH_RATIO"][28])


fig, ax = plt.subplots(1)

plt.xlim(-87.9,-87.5)
plt.ylim(41.6,42.05)

ax.add_patch(region1)
ax.add_patch(region2)
ax.add_patch(region3)
ax.add_patch(region4)
ax.add_patch(region5)
ax.add_patch(region6)
ax.add_patch(region7)
ax.add_patch(region8)
ax.add_patch(region9)
ax.add_patch(region10)
ax.add_patch(region11)
ax.add_patch(region12)
ax.add_patch(region13)
ax.add_patch(region14)
ax.add_patch(region15)
ax.add_patch(region16)
ax.add_patch(region17)
ax.add_patch(region18)
ax.add_patch(region19)
ax.add_patch(region20)
ax.add_patch(region21)
ax.add_patch(region22)
ax.add_patch(region23)
ax.add_patch(region24)
ax.add_patch(region25)
ax.add_patch(region26)
ax.add_patch(region27)
ax.add_patch(region28)
ax.add_patch(region29)


ax.set_aspect('equal', adjustable='box')
ax.set_axis_off()

plt.title("Crashes by Area per \nEB-Estimate for " + fatality + " / " + route)

plt.savefig("Fig_EB" + type + ".png", bbox_inches='tight')
plt.show()